<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
# !wget "https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1"

In [2]:
from gensim.models.keyedvectors import KeyedVectors

wv_embeddings = KeyedVectors.load_word2vec_format(
    "SO_vectors_200.bin?download=1", binary=True
)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = "dog"
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли словo `cat` в топ-5 близких слов к слову `dog`? Какое место? 

In [5]:
# method most_simmilar
wv_embeddings.most_similar(positive="dog", topn=5)

[('animal', 0.8564179539680481),
 ('dogs', 0.7880867123603821),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.7607938647270203)]

#### Ответ:
Да, входит. Стоит на 4-м месте. 

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [6]:
import re
import string

import numpy as np
from nltk import SnowballStemmer
from nltk.corpus import stopwords

stopWords = set(stopwords.words("english"))
stemmer = SnowballStemmer(language="english")


class MyTokenizer:
    def __init__(self):
        pass

    def tokenize(self, text):
        text = text.lower()
        text = re.sub(f"[']", "", text)
        text = re.sub(f"[{string.punctuation}]", " ", text)
        text = re.sub("[^a-z]", " ", text)
        text = re.findall("[a-z]+", text)
        text = [word for word in text if word not in stopWords]
        return list(map(stemmer.stem, text))


tokenizer = MyTokenizer()

In [7]:
st = "Oh, I think I've landed 43 Where there are miracles at work,  For the thirst and for the hunger Come the conference of birds"
tokenizer.tokenize(st)

['oh',
 'think',
 'ive',
 'land',
 'miracl',
 'work',
 'thirst',
 'hunger',
 'come',
 'confer',
 'bird']

In [8]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
    question: строка
    embeddings: наше векторное представление
    dim: размер любого вектора в нашем представлении

    return: векторное представление для вопроса
    """
    found_embeds = 0
    a = np.empty((0, dim))
    for token in tokenizer.tokenize(question):
        if token in embeddings:
            a = np.append(a, [embeddings[token]], axis=0)
            found_embeds += 1
    if found_embeds > 0:
        return np.mean(a, axis=0)
    else:
        return np.zeros(dim)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [9]:
vec = question_to_vec("I love neural networks", wv_embeddings, tokenizer)
vec[2].round(2)

-1.41

#### Ответ:
3-я компонента имеет значение $\approx -1.41$

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

#### Oтвет

Чтобы получить максимум разници, нужно из максимально большого уменьшаемого вычесть максимально маленькое вычитаемое:
$$\text{MAX}(\text{Hits@47} - \text{DCG@1}) = \text{MAX}(\text{Hits@47}) - \text{MIN}(\text{DCG@1})$$

Минимальное $DCG@1$ возможно когда ответ не стоит на первом месте в выдаче $rank\_q_{i} > 1$:
$$\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$$

Тогда, условием для получения максимально значения будет:
$$1 < rank\_q_{i}' \le 47$$


И ответом будет $1$:
$$
\text{MAX}(\text{Hits@47} - \text{DCG@1})
= \text{MAX}(\text{Hits@47}) - \text{MIN}(\text{DCG@1})
= 1 - 0 
= \boldsymbol{1}
$$

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

read_corpus Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

#### Oтвет:

$$\text{DCG@10} = \frac{1}{\log_2(1+9)}\cdot[9 \le 10] \approx 0.3$$

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [10]:
def hits_count(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов
    result: вернуть  Hits@k
    """
    dup_ranks = np.array(dup_ranks)
    hits_value = np.mean(dup_ranks <= k)
    return hits_value

In [11]:
def dcg_score(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов
    result: вернуть DCG@k
    """
    dup_ranks = np.array(dup_ranks)
    dcg_value = np.mean(1 / np.log2(1 + dup_ranks) * (dup_ranks <= k).astype(int))
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [12]:
import pandas as pd

In [13]:
copy_answers = [
    "How does the catch keyword determine the type of exception that was thrown",
]

# наги кандидаты
candidates_ranking = [
    [
        "How Can I Make These Links Rotate in PHP",
        "How does the catch keyword determine the type of exception that was thrown",
        "NSLog array description not memory address",
        "PECL_HTTP not recognised php ubuntu",
    ],
]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print("Ваш ответ HIT:", [hits_count(dup_ranks, k) for k in range(1, 5)])
print("Ваш ответ DCG:", [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [14]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame(
    [[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
    index=["HITS", "DCG"],
    columns=range(1, 5),
)
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [15]:
# !unzip stackoverflow_similar_questions.zip

Считайте данные.

In [16]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding="utf-8"):
        data.append(line.split("\t"))
    return data

Нам понадобится только файл validation.

In [17]:
validation_data = read_corpus("./data/validation.tsv")

Кол-во строк

In [18]:
len(validation_data)

3760

Размер нескольких первых строк

In [19]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [20]:
from copy import deepcopy

from sklearn.metrics.pairwise import cosine_similarity

In [21]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
    question: строка
    candidates: массив строк(кандидатов) [a, b, c]
    result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    X = question_to_vec(question, embeddings, tokenizer).reshape(1, -1)

    Ys = [question_to_vec(y, embeddings, tokenizer).reshape(1, -1) for y in candidates]

    cosines = []
    ranks = []

    for i in range(len(Ys)):

        cosines.append(cosine_similarity(X, Ys[i])[0][0])
        ranks.append((i, candidates[i]))

    ids = np.argsort(cosines)[::-1]
    result = [ranks[i] for i in ids]

    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [22]:
questions = ["converting string to list", "Sending array via Ajax fails"]

candidates = [
    [
        "Convert Google results object (pure js) to Python object",  # первый эксперимент
        "C# create cookie from string and send it",
        "How to use jQuery AJAX for an outside domain?",
    ],
    [
        "Getting all list items of an unordered list in PHP",  # второй эксперимент
        "WPF- How to update the changes in list item of a list",
        "select2 not displaying search results",
    ],
]

In [23]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
    [print(rank) for rank in ranks]
    print()

(1, 'C# create cookie from string and send it')
(0, 'Convert Google results object (pure js) to Python object')
(2, 'How to use jQuery AJAX for an outside domain?')

(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')
(1, 'WPF- How to update the changes in list item of a list')



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [24]:
# должно вывести
results = [
    [
        (1, "C# create cookie from string and send it"),
        (0, "Convert Google results object (pure js) to Python object"),
        (2, "How to use jQuery AJAX for an outside domain?"),
    ],
    [
        ("*", "Getting all list items of an unordered list in PHP"),  # скрыт
        ("*", "select2 not displaying search results"),  # скрыт
        ("*", "WPF- How to update the changes in list item of a list"),
    ],
]  # скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

***

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

#### Ответ

Последовательность для второго эксперимента получилась: `0-2-1`

***

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [25]:
from tqdm import tqdm

In [26]:
np.random.seed(42)
max_validation_examples = 1000

ids = np.random.randint(0, len(validation_data), size=max_validation_examples)
val_data_sample = [validation_data[i] for i in ids]
wv_ranking = []

for i, line in enumerate(tqdm(val_data_sample)):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 1000/1000 [02:04<00:00,  8.01it/s]


In [27]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k))
    )

100%|██████████| 6/6 [00:00<00:00, 5137.98it/s]

DCG@   1: 0.375 | Hits@   1: 0.375
DCG@   5: 0.456 | Hits@   5: 0.530
DCG@  10: 0.478 | Hits@  10: 0.598
DCG@ 100: 0.522 | Hits@ 100: 0.816
DCG@ 500: 0.539 | Hits@ 500: 0.950
DCG@1000: 0.544 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [28]:
train_data = read_corpus("./data/train.tsv")

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [29]:
sentences = [tokenizer.tokenize(" ".join(pair)) for pair in tqdm(train_data)]

100%|██████████| 1000000/1000000 [01:26<00:00, 11571.08it/s]


In [30]:
from gensim.models import Word2Vec

In [31]:
embeddings_trained = Word2Vec(
    sentences,  # data for model to train on
    vector_size=200,  # embedding vector size
    min_count=5,  # consider words that occured at least 5 times
    window=20,
    workers=8,
).wv

In [32]:
wv_ranking = []
for i, line in enumerate(tqdm(val_data_sample)):
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

100%|██████████| 1000/1000 [02:04<00:00,  8.00it/s]


In [33]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k))
    )

100%|██████████| 6/6 [00:00<00:00, 8981.38it/s]

DCG@   1: 0.467 | Hits@   1: 0.467
DCG@   5: 0.562 | Hits@   5: 0.643
DCG@  10: 0.582 | Hits@  10: 0.705
DCG@ 100: 0.621 | Hits@ 100: 0.892
DCG@ 500: 0.632 | Hits@ 500: 0.980
DCG@1000: 0.635 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


- В данной задаче лучше сработала токенизация с исключением пунктуационных знаков и цифр, а также приведением всех букв к нижнему регистру. Видимо, перечисленные части текста дают больше информационного шума чем персказательной способности. Также стоит отметить, что исключение стоп-слов — наоборот — ухудшило скоринг на этой задаче.
- Лемматицация слов помогла значительно улучшить результаты.
- Эмбеддинги с бóльшим размером окна справляются с задачей лучше. Вероятно, это связано со спосбностью улавливать взаимосвязь даже с далеко стоящими в предложении словами.
- Подход word2vec в разобранной задаче ограничен текущим словарем и не учитывает более сложных взаимоотношений между словами.
- Для решения этой задаче можно было бы, например, использовать предобученную модель BERT с дообучением на тренировочном датасете.